## Requirements

In [ ]:
import scipy.integrate
from vpython import canvas, color, cylinder, rate, sphere, sin, cos, pi, vector

## Set of ordinary differential equations

The motion of a frictionless pendulum is given by the following set of first order differentical equations:

$$
    \begin{cases}
        \frac{d \theta}{d t}(t) = \omega(t) \\
        \frac{d \omega}{d t}(t) = -\frac{m g}{l} \sin \theta(t)
    \end{cases}
$$

where $m$ and $l$ are the mass and the length of the pendulum respectively, $g$ is the gravitational accelleration, $theta(t)$ is the angle of the pendulum with respect to the $Y$-axis, and $\omega(t)$ is the angular velocity.

To numerically solve this set of equations, we write a function that returns the values of the righthand side of these equations as a list.

In [ ]:
def funcs(t, Y, g, l, m):
    theta, omega = Y
    return [
        omega,
        -m*g*sin(theta)/l,
    ]

In addition, we compute the Jacobian, i.e., if we define $f_1(\theta, \omega, t) = \omega$ and $f_2(\theta, \omega, t) = - \frac{m g}{l} \sin \theta$, than the Jacobian is given by:
$$
    \left(
        \begin{matrix}
            \frac{\partial f_1}{\partial \theta} & \frac{\partial f_1}{\partial \omega} \\
            \frac{\partial f_2}{\partial \theta} & \frac{\partial f_2}{\partial \omega}
        \end{matrix}
    \right)
$$

In [ ]:
def jac(t, Y, g, l, m):
    theta, omega = Y
    return [
        [0.0, 1.0],
        [-m*g*cos(theta)/l, 0.0],
    ]

Next, we define a function that takes all relevant parameters as arguments, sets up the VPython scene, the system of ODEs, and integrates the set of equations, updating the scene's objects after each time step.

In [ ]:
def animate(t_max, theta0, omega0=0.0, g=9.81, l=9.81/10.0, m=1.0, delta_t=0.01):
    rate_value = int(1.0/delta_t)
    # set up the scene
    canvas()
    # set a hidden point in the scene to make sure that the pivot is centered
    hidden = sphere(pos=vector(-l/2.0, -l, 0.0), radius=0.0001)
    pivot = sphere(pos=vector(0.0, 0.0, 0.0), radius=0.005)
    mass = sphere(pos=vector(l*sin(theta0), -l*cos(theta0), 0.0),
                  radius=0.05,
                  color=color.yellow,
                  make_trail=True, retain=10)
    string = cylinder(pos=pivot.pos, axis=mass.pos - pivot.pos,
                      radius= 0.01)
    # define the ODE system
    system = scipy.integrate.ode(funcs, jac) \
        .set_integrator('dopri5') \
        .set_f_params(g, l, m) \
        .set_jac_params(g, l, m) \
        .set_initial_value((theta0, omega0), 0.0)
    # integrate the system of ODEs and update the scene's objects accordingly
    while system.successful() and system.t < t_max:
        rate(rate_value)
        system.integrate(system.t + delta_t)
        theta, omega = system.y
        mass.pos = vector(l*sin(theta), -l*cos(theta), 0.0)
        string.axis =  mass.pos - pivot.pos
    mass.clear_trail()

Run the animation for $\theta_0 = \frac{\pi}{6}$.

In [ ]:
animate(10*pi, pi/6.0)

Run the animation for $\theta = \frac{95\pi}{100}$.

In [ ]:
animate(10*pi, 0.95*pi)